Import Libraries

In [2]:
import pandas as pd 
import os
import shutil
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import PIL
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import sklearn
from sklearn import metrics

In [4]:
folder_path = 'model2_data'
# train_path = folder_path+'/train'
# val_path = folder_path+'/valid'
# test_path = folder_path+'/test'
# train_csv = pd.read_csv(train_path+'/_classes.csv')
# val_csv = pd.read_csv(val_path+'/_classes.csv')
# test_csv = pd.read_csv(test_path+'/_classes.csv')
# frame = pd.concat((train_csv,val_csv,test_csv), axis=0, ignore_index=True)
# frame.to_csv(folder_path+'/classes.csv',index=False)
# files = os.listdir(train_path)
# shutil.copytree(train_path, folder_path+'/allData')
# shutil.copytree(test_path, folder_path+'/allData')
# shutil.copytreev(val_path, folder_path+'/allData')

In [5]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, images_folder, transform = None):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        filename = self.df.loc[index]['filename']
        labels = self.df.iloc[index][1:].values.astype(float)
        image = PIL.Image.open(os.path.join(self.images_folder, filename))
        if self.transform is not None:
            image = self.transform(image)
        return image, torch.tensor(labels)

In [19]:
test_path = folder_path + '/testData'
data_path = folder_path + '/allData'
df = pd.read_csv(folder_path+'/classes.csv')
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
dataset = CustomDataset(folder_path+'/classes.csv', data_path, transform=transform)
testDataset = datasets.ImageFolder(test_path,transform=transform)
print(testDataset.classes)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.9, 0.1])

['OK', 'blobs', 'spaghetti', 'stringing']


In [21]:
num_of_classes = 4
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_of_classes)

for param in model.parameters():
    print(param.requires_grad)


c:\Users\aalqa\anaconda3\envs\cs323\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\aalqa\anaconda3\envs\cs323\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True


In [47]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

trainLoader =  DataLoader(dataset, batch_size=32, shuffle=True)
testLoader =  DataLoader(test_dataset, batch_size=32, shuffle=False)
test2Loader =  DataLoader(testDataset, batch_size=32, shuffle=False)


In [48]:
for epoch in range(12):

    running_loss = 0.0
    train_loss_history = []
    for i, data in enumerate(trainLoader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 10 == 5:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

c:\Users\aalqa\anaconda3\envs\cs323\lib\site-packages\torch\autograd\graph.py:768: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[1,     6] loss: 0.004
[1,    16] loss: 0.007
[1,    26] loss: 0.006
[1,    36] loss: 0.005
[1,    46] loss: 0.004
[1,    56] loss: 0.004
[2,     6] loss: 0.002
[2,    16] loss: 0.003
[2,    26] loss: 0.003
[2,    36] loss: 0.003
[2,    46] loss: 0.003
[2,    56] loss: 0.003
[3,     6] loss: 0.002
[3,    16] loss: 0.003
[3,    26] loss: 0.003
[3,    36] loss: 0.003
[3,    46] loss: 0.003
[3,    56] loss: 0.003
[4,     6] loss: 0.001
[4,    16] loss: 0.002
[4,    26] loss: 0.002
[4,    36] loss: 0.002
[4,    46] loss: 0.002
[4,    56] loss: 0.003
[5,     6] loss: 0.001
[5,    16] loss: 0.002
[5,    26] loss: 0.002
[5,    36] loss: 0.002
[5,    46] loss: 0.002
[5,    56] loss: 0.002
[6,     6] loss: 0.001
[6,    16] loss: 0.002
[6,    26] loss: 0.002
[6,    36] loss: 0.002
[6,    46] loss: 0.002
[6,    56] loss: 0.002
[7,     6] loss: 0.001
[7,    16] loss: 0.002
[7,    26] loss: 0.002
[7,    36] loss: 0.002
[7,    46] loss: 0.002
[7,    56] loss: 0.002
[8,     6] loss: 0.001
[8,    16] 

In [49]:
model.eval()

correct = 0
total = 0
all_labels = []
all_preds = []
with torch.no_grad():
    for inputs, labels in test2Loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        modified_tensor = labels.clone()
        modified_tensor[labels == 1] = 3
        modified_tensor[labels == 2] = 1
        modified_tensor[labels == 3] = 2
        real = modified_tensor
        print(predicted,"with",modified_tensor)
        total += labels.size(0)
        all_labels.extend(real.numpy())
        all_preds.extend(predicted.numpy())
        correct += (predicted == real).sum().item()

accuracy = correct / total * 100
print(f'Test Accuracy: {accuracy:.2f}%')
        

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 3, 0, 0, 0]) with tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]) with tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]) with tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0,
        3, 0, 0, 3, 3, 3, 3, 3]) with tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 3, 3, 3, 3, 3])
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [50]:
report = metrics.classification_report(all_labels, all_preds, target_names=['OK','Spaghetti','stringing','blobs'])
print(report)

              precision    recall  f1-score   support

          OK       0.92      0.93      0.92       123
   Spaghetti       1.00      0.87      0.93       100
   stringing       0.80      0.80      0.80        86
       blobs       0.76      0.96      0.85        47

    accuracy                           0.88       356
   macro avg       0.87      0.89      0.88       356
weighted avg       0.89      0.88      0.89       356

